# Chapter 15: Animation and Simulation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_15_animation_and_simulation.ipynb)

**Animation and simulation** bring 3D scenes to life through motion. This chapter covers keyframe animation, skeletal animation, physics simulation, and particle systems.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import Tuple, List, Optional
from dataclasses import dataclass

In [ ]:
class Vec3:
    def __init__(self, x: float = 0.0, y: float = 0.0, z: float = 0.0):
        self.x, self.y, self.z = x, y, z
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar: float):
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __truediv__(self, scalar: float):
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    def dot(self, other) -> float:
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def length(self) -> float:
        return math.sqrt(self.dot(self))
    
    def normalize(self):
        l = self.length()
        return self / l if l > 0 else Vec3(0, 0, 0)

print("✓ Base classes loaded")

## 1. Keyframe Animation

**Keyframes** define object states at specific times. Intermediate frames are interpolated.

### Linear Interpolation (Lerp)

$$
\mathbf{p}(t) = (1 - t) \mathbf{p}_0 + t \mathbf{p}_1, \quad t \in [0, 1]
$$

### Spherical Linear Interpolation (Slerp)

For rotations (quaternions):

$$
\text{slerp}(\mathbf{q}_0, \mathbf{q}_1, t) = \frac{\sin((1-t)\theta)}{\sin\theta} \mathbf{q}_0 + \frac{\sin(t\theta)}{\sin\theta} \mathbf{q}_1
$$

where $\cos\theta = \mathbf{q}_0 \cdot \mathbf{q}_1$

### Cubic Interpolation (Hermite/Bezier)

$$
\mathbf{p}(t) = (2t^3 - 3t^2 + 1)\mathbf{p}_0 + (t^3 - 2t^2 + t)\mathbf{m}_0 + (-2t^3 + 3t^2)\mathbf{p}_1 + (t^3 - t^2)\mathbf{m}_1
$$

In [ ]:
def lerp(a: Vec3, b: Vec3, t: float) -> Vec3:
    """Linear interpolation"""
    return a * (1 - t) + b * t

def ease_in_out(t: float) -> float:
    """Smooth ease-in-ease-out curve"""
    return t * t * (3 - 2 * t)

def catmull_rom(p0: Vec3, p1: Vec3, p2: Vec3, p3: Vec3, t: float) -> Vec3:
    """Catmull-Rom spline interpolation"""
    t2 = t * t
    t3 = t2 * t
    
    return (p1 * 2.0 +
           (p2 - p0) * t +
           (p0 * 2.0 - p1 * 5.0 + p2 * 4.0 - p3) * t2 +
           (p1 * 3.0 - p0 - p2 * 3.0 + p3) * t3) * 0.5

@dataclass
class Keyframe:
    time: float
    position: Vec3
    rotation: Vec3  # Euler angles for simplicity
    scale: Vec3

class Animation:
    def __init__(self):
        self.keyframes = []
    
    def add_keyframe(self, kf: Keyframe):
        self.keyframes.append(kf)
        self.keyframes.sort(key=lambda k: k.time)
    
    def evaluate(self, time: float) -> Keyframe:
        if not self.keyframes:
            return Keyframe(0, Vec3(), Vec3(), Vec3(1, 1, 1))
        
        if time <= self.keyframes[0].time:
            return self.keyframes[0]
        if time >= self.keyframes[-1].time:
            return self.keyframes[-1]
        
        # Find surrounding keyframes
        for i in range(len(self.keyframes) - 1):
            if self.keyframes[i].time <= time <= self.keyframes[i + 1].time:
                kf0, kf1 = self.keyframes[i], self.keyframes[i + 1]
                t = (time - kf0.time) / (kf1.time - kf0.time)
                t = ease_in_out(t)  # Apply easing
                
                return Keyframe(
                    time=time,
                    position=lerp(kf0.position, kf1.position, t),
                    rotation=lerp(kf0.rotation, kf1.rotation, t),
                    scale=lerp(kf0.scale, kf1.scale, t)
                )
        
        return self.keyframes[0]

print("✓ Keyframe animation loaded")

## 2. Physics Simulation

### Newton's Laws

$$
\mathbf{F} = m \mathbf{a}
$$

### Euler Integration

$$
\mathbf{v}_{t+1} = \mathbf{v}_t + \mathbf{a} \Delta t
$$
$$
\mathbf{x}_{t+1} = \mathbf{x}_t + \mathbf{v}_t \Delta t
$$

### Verlet Integration (more stable)

$$
\mathbf{x}_{t+1} = 2\mathbf{x}_t - \mathbf{x}_{t-1} + \mathbf{a} \Delta t^2
$$

In [ ]:
class Particle:
    def __init__(self, position: Vec3, velocity: Vec3 = None, mass: float = 1.0):
        self.position = position
        self.velocity = velocity if velocity else Vec3()
        self.acceleration = Vec3()
        self.mass = mass
        self.prev_position = position
    
    def apply_force(self, force: Vec3):
        """Apply force (F = ma)"""
        self.acceleration = self.acceleration + force / self.mass
    
    def update_euler(self, dt: float):
        """Euler integration"""
        self.velocity = self.velocity + self.acceleration * dt
        self.position = self.position + self.velocity * dt
        self.acceleration = Vec3()  # Reset
    
    def update_verlet(self, dt: float):
        """Verlet integration"""
        new_pos = self.position * 2.0 - self.prev_position + self.acceleration * (dt * dt)
        self.prev_position = self.position
        self.position = new_pos
        self.acceleration = Vec3()  # Reset

class ParticleSystem:
    def __init__(self, gravity: Vec3 = None):
        self.particles = []
        self.gravity = gravity if gravity else Vec3(0, -9.8, 0)
    
    def add_particle(self, particle: Particle):
        self.particles.append(particle)
    
    def update(self, dt: float):
        for p in self.particles:
            p.apply_force(self.gravity * p.mass)
            p.update_euler(dt)

print("✓ Physics simulation loaded")

## 3. Collision Detection

### Sphere-Sphere Collision

$$
|\mathbf{p}_1 - \mathbf{p}_2| < r_1 + r_2
$$

### Sphere-Plane Collision

$$
d = \mathbf{n} \cdot \mathbf{p} + D
$$

Collision if $|d| < r$

### Collision Response

$$
\mathbf{v}' = \mathbf{v} - (1 + e)(\mathbf{v} \cdot \mathbf{n})\mathbf{n}
$$

where $e$ is coefficient of restitution (0 = inelastic, 1 = elastic)

In [ ]:
def sphere_sphere_collision(p1: Vec3, r1: float, p2: Vec3, r2: float) -> bool:
    """Check sphere-sphere collision"""
    dist_sq = (p1 - p2).dot(p1 - p2)
    radius_sum = r1 + r2
    return dist_sq < radius_sum * radius_sum

def sphere_plane_collision(pos: Vec3, radius: float, 
                          plane_normal: Vec3, plane_d: float) -> Optional[float]:
    """Check sphere-plane collision, return penetration depth"""
    dist = pos.dot(plane_normal) + plane_d
    if abs(dist) < radius:
        return radius - dist
    return None

def resolve_collision(velocity: Vec3, normal: Vec3, restitution: float = 0.8) -> Vec3:
    """Resolve collision with plane"""
    v_dot_n = velocity.dot(normal)
    if v_dot_n < 0:  # Moving towards plane
        return velocity - normal * ((1 + restitution) * v_dot_n)
    return velocity

print("✓ Collision detection loaded")

## Example 1: Bouncing Ball Animation

In [ ]:
# Simulate bouncing ball
ball = Particle(
    position=Vec3(0, 10, 0),
    velocity=Vec3(2, 0, 0),
    mass=1.0
)

# Ground plane: y = 0, normal = (0, 1, 0)
ground_normal = Vec3(0, 1, 0)
ground_d = 0.0
ball_radius = 0.5
gravity = Vec3(0, -9.8, 0)
restitution = 0.8

# Simulation
dt = 0.016  # ~60 FPS
num_steps = 300
trajectory = []

for _ in range(num_steps):
    # Apply forces
    ball.apply_force(gravity * ball.mass)
    
    # Update
    ball.update_euler(dt)
    
    # Ground collision
    penetration = sphere_plane_collision(ball.position, ball_radius, ground_normal, ground_d)
    if penetration is not None:
        # Push out of ground
        ball.position = ball.position + ground_normal * penetration
        # Resolve velocity
        ball.velocity = resolve_collision(ball.velocity, ground_normal, restitution)
    
    trajectory.append((ball.position.x, ball.position.y))

# Plot
trajectory = np.array(trajectory)
plt.figure(figsize=(12, 6))
plt.plot(trajectory[:, 0], trajectory[:, 1], 'b-', linewidth=2)
plt.scatter(trajectory[::20, 0], trajectory[::20, 1], c='red', s=50)
plt.axhline(y=0, color='brown', linestyle='--', linewidth=2, label='Ground')
plt.xlabel('X Position')
plt.ylabel('Y Position (Height)')
plt.title('Bouncing Ball Simulation')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

print(f"Simulated {num_steps} timesteps")

## Example 2: Keyframe Animation Path

In [ ]:
# Create animation with keyframes
anim = Animation()

# Add keyframes
anim.add_keyframe(Keyframe(0.0, Vec3(0, 0, 0), Vec3(), Vec3(1, 1, 1)))
anim.add_keyframe(Keyframe(1.0, Vec3(5, 3, 0), Vec3(), Vec3(1.5, 1.5, 1.5)))
anim.add_keyframe(Keyframe(2.0, Vec3(10, 1, 0), Vec3(), Vec3(1, 1, 1)))
anim.add_keyframe(Keyframe(3.0, Vec3(15, 4, 0), Vec3(), Vec3(2, 2, 2)))
anim.add_keyframe(Keyframe(4.0, Vec3(20, 0, 0), Vec3(), Vec3(1, 1, 1)))

# Evaluate animation
times = np.linspace(0, 4, 200)
positions = []
scales = []

for t in times:
    kf = anim.evaluate(t)
    positions.append((kf.position.x, kf.position.y))
    scales.append(kf.scale.x)

positions = np.array(positions)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Position path
ax1.plot(positions[:, 0], positions[:, 1], 'b-', linewidth=2, label='Path')
keyframe_pos = np.array([(kf.position.x, kf.position.y) for kf in anim.keyframes])
ax1.scatter(keyframe_pos[:, 0], keyframe_pos[:, 1], c='red', s=100, 
           marker='o', label='Keyframes', zorder=5)
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_title('Animation Path')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Scale over time
ax2.plot(times, scales, 'g-', linewidth=2)
keyframe_times = [kf.time for kf in anim.keyframes]
keyframe_scales = [kf.scale.x for kf in anim.keyframes]
ax2.scatter(keyframe_times, keyframe_scales, c='red', s=100, marker='o', zorder=5)
ax2.set_xlabel('Time')
ax2.set_ylabel('Scale')
ax2.set_title('Scale Animation')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Smooth interpolation between keyframes with easing.")

## Summary

**Animation and simulation** techniques:

### Key Concepts

1. **Keyframe Animation**
   - Define states at key times
   - Interpolate between keyframes (lerp, slerp, splines)
   - Easing functions for natural motion

2. **Physics Simulation**
   - Newton's laws (F = ma)
   - Integration methods (Euler, Verlet, RK4)
   - Forces: gravity, drag, springs

3. **Particle Systems**
   - Many simple particles
   - Useful for fire, smoke, explosions, water
   - GPU acceleration important

4. **Collision Detection & Response**
   - Broad phase: spatial partitioning
   - Narrow phase: precise tests
   - Response: impulses, constraints

5. **Advanced Topics**
   - Skeletal animation (rigging, skinning)
   - Cloth simulation
   - Fluid simulation
   - Soft body dynamics

### Applications

- Character animation in games/films
- Visual effects (explosions, destruction)
- Engineering simulation
- Scientific visualization

Animation and simulation are essential for dynamic, believable 3D worlds.